# Cleaning and Formatting Protein Mutation Data

## ProThermDB 

## Dataset Stats

Number of Entries = 31,470

#### Importing Data and Dropping Irrelevant Columns

In [177]:
import os
import numpy as np
import pandas as pd

# Jupyter notebook settings to see all data
max_rows = None
max_cols = None
pd.set_option("display.max_rows", max_rows, "display.max_columns", max_cols) 

# load unmodified dataset
protherm_path = os.path.join("DBdata", "ProThermDB_30_JUN_21.tsv")
protherm_df = pd.read_csv(protherm_path,sep='\t')

# removes unneccessary columns
col_to_drop = ["NO", "SEC_STR", "ASA", "KEY_WORDS", "AUTHOR", "REMARKS"]
protherm_df = protherm_df.drop(columns=col_to_drop)

print(protherm_df.columns) 

# add dataset label
protherm_df.insert(0, "DATABASE", 'Protherm')


Index(['PROTEIN', 'UniProt_ID', 'MUTATION', 'SOURCE', 'PDB_wild',
       'PDB_Chain_Mutation', 'pH', 'T_(C)', 'Tm_(C)', '∆Tm_(C)',
       '∆H_(kcal/mol)', '∆G_(kcal/mol)', '∆∆G_(kcal/mol)',
       '∆∆G_H2O_(kcal/mol)', 'STATE', 'REVERSIBILITY', 'PubMed_ID',
       'REFERENCE'],
      dtype='object')


#### Functions used to Clean and Cast Column values

In [178]:
# function casts all values in specified columns to floats
def cast_to_float(df, col_name_list):
    for col in col_name_list:
        df[col] = df[col].astype(float)
    return df

# function casts all values in specified columns to strings
def cast_to_string(df, col_name_list):
    for col in col_name_list:
        df[col] = np.where(pd.isnull(df[col]),df[col],df[col].astype(str))
    return df

# function returns list w/ only mean values of column (e.g 50 (0.01) -> 50)
def extract_mean_val_list(column):
    column = column.fillna("-")
    values = column.str.split(" ")
    mean_vals = []
    for val in values:
        mean = val[0].strip(" ")
        if "(" in mean:
            mean = mean[:mean.index("(")]
        if ">" in mean or "<" in mean:
            mean = "-"
        if len(mean) == 0:
            mean = "-"
        if "-" in mean and mean.index("-") != 0:
            nums = mean.split("-")
            nums = [float(i) for i in nums]
            mean = (nums[0] + nums[1]) / 2
        mean_vals.append(mean)
    return mean_vals


#### Cleaning dataframe columns using above functions

In [179]:
# removes all error values from columns w/ experimental error data included
protherm_df["Tm_(C)"] = extract_mean_val_list(protherm_df["Tm_(C)"])
protherm_df["∆Tm_(C)"] = extract_mean_val_list(protherm_df["∆Tm_(C)"])
protherm_df["∆H_(kcal/mol)"] = extract_mean_val_list(protherm_df["∆H_(kcal/mol)"])
protherm_df["∆G_(kcal/mol)"] = extract_mean_val_list(protherm_df["∆G_(kcal/mol)"])
protherm_df["∆∆G_(kcal/mol)"] = extract_mean_val_list(protherm_df["∆∆G_(kcal/mol)"])
protherm_df["∆∆G_H2O_(kcal/mol)"] = extract_mean_val_list(protherm_df["∆∆G_H2O_(kcal/mol)"])

# remove "(Based on UniProt)"  or "(Based on UniProt and PBD)" from MUTATION column
split_mutation_col = protherm_df["MUTATION"].str.split("(")
names = []
for string in split_mutation_col:
    mutation_name = string[0].strip(" ")
    names.append(mutation_name)
protherm_df["MUTATION"] = names

# remove PMID from REFERENCE
fixed_reference = []
for string in protherm_df["REFERENCE"]:
    pmid_index = string.find("PMID")
    fixed_reference.append(string[0:pmid_index-1])
protherm_df["REFERENCE"] = fixed_reference

# clear error DOI strings from "PubMed_ID" colum and cast to float
correct_ids = []
for value in protherm_df["PubMed_ID"]:
    if "DOI" in value or "(" in value:
        value = "-"
    correct_ids.append(value)
protherm_df["PubMed_ID"] = correct_ids      

# creating MUTATED_CHAIN column
protherm_df["MUTATED_CHAIN"] = np.nan

# temporarily replace NaN with "-" to avoid errors
protherm_df["PDB_Chain_Mutation"] = protherm_df["PDB_Chain_Mutation"].fillna("-")

# extract mutated chain from PBD_CHAIN_MUTATION and add to MUTATED_CHAIN col
protherm_df["PDB_Chain_Mutation"] = protherm_df["PDB_Chain_Mutation"].str.strip(" ")
split_chain_col = protherm_df["PDB_Chain_Mutation"].str.split(" ")
chain = []
chain_letters = []
for string in split_chain_col:
    if "-" in string:
        to_add_chains = np.nan
    else:
        for val in string:
            index = val.find("_")
            letter = val[index+1]
            chain_letters.append(letter)
            to_add_chains = ",".join(chain_letters)
    chain.append(to_add_chains)
    chain_letters.clear()
protherm_df["MUTATED_CHAIN"] = chain

# replace all "-" values with NaN
protherm_df = protherm_df.replace("-", np.nan)

# cast experimental values + PubMed_ID to floats
protherm_df = cast_to_float(protherm_df, ["pH", "T_(C)", "Tm_(C)", "∆Tm_(C)", "∆H_(kcal/mol)",
                                          "∆G_(kcal/mol)", "∆∆G_(kcal/mol)", "∆∆G_H2O_(kcal/mol)",
                                          "PubMed_ID"])


## ThermoMutDB

Number of Entries = 13,348

In [180]:
# load unmodified dataset
thermo_path = os.path.join("DBdata", "thermomutdb.csv")
thermo_df = pd.read_csv(thermo_path)

# removing unnecessary columns and add DATABASE column
col_to_drop = ["ID", "YEAR", "DOI", "OTHER_LINKS", "HYDRO",
               "POS", "NEG", "ACC", "DON", "ARO", "SUL", "NEU", "SST", "RSA", "PHI",
               "PSI","RES_DEPTH", "CA_DEPTH", "RELATIVE_BFACTOR", "BLOSUM62", "PAM250",
               "SEQ_Uniprot", "MUTATION_uniprot", "MUTATION_pdb"]

thermo_df = thermo_df.drop(columns=col_to_drop)
thermo_df.insert(0, "DATABASE", 'ThermoMut')

# convert temperature units / dTM units to match protherm db (K to C)
thermo_df["TEMP"] = thermo_df["TEMP"] - 273.15

# renaming and adding columns to match formatting of final dataset
dictionary = {"UNIPROT": "UniProt_ID",
        "TEMP": "T_(C)",
        "ddG":"∆∆G_(kcal/mol)",
        "dTm": "∆Tm_(C)",
        "PMID": "PubMed_ID"}

thermo_df.rename(columns=dictionary, inplace = True)
print(thermo_df.columns)


Index(['DATABASE', 'PROTEIN', 'SOURCE', 'KINGDOM', 'UniProt_ID', 'PDB_wild',
       'PDB_mutant', 'MUTATION', 'MUTATED_CHAIN', 'T_(C)', 'pH', 'MEASURE',
       'METHOD', '∆∆G_(kcal/mol)', '∆Tm_(C)', 'REFERENCE', 'PubMed_ID'],
      dtype='object')


## FireProtDB

Number of Entries = 17,136

In [181]:
# loading data
fireprot_path = os.path.join("DBdata", "fireprotdb_results.csv")
fireprot_df = pd.read_csv(fireprot_path, dtype="unicode")

# dropping unneccessary cols
col_to_drop = ["experiment_id", "type", "derived_type", "is_essential", "secondary_structure", "asa", "b_factor",
               "method_details", "technique_details", "publication_doi", "hsw_job_id", "sequence", 
               "interpro_families", "correlated_pairs", "back_to_consensus", "is_in_catalytic_pocket",
               "tunnels", "is_in_bottleneck"]

fireprot_df = fireprot_df.drop(columns=col_to_drop)

# inserting database labels
fireprot_df.insert(0, "DATABASE", "FireProt")

# renaming columns
dictionary = {"protein_name": "PROTEIN",
              "uniprot_id": "UniProt_ID",
              "chain": "MUTATED_CHAIN",
              "wild_type": "WILD_TYPE_RES",
              "mutation": "MUTATED_RES",
              "position": "POSITION",
              "method": "METHOD",
              "ddG": "∆∆G_H2O_(kcal/mol)",
              "dTm": "∆Tm_(C)",
              "tm": "Tm_(C)",
              "publication_pubmed": "PubMed_ID",
              "pdb_id": "PDB_wild",
              "technique": "MEASURE"}

fireprot_df.rename(columns=dictionary, inplace = True)

# drop exact duplicates
fireprot_df.drop_duplicates(keep="first", inplace=True)

# cast experimental data columns to float
fireprot_df = cast_to_float(fireprot_df, ["pH", "Tm_(C)", "∆Tm_(C)",
                                          "∆∆G_H2O_(kcal/mol)",
                                          "PubMed_ID", "POSITION", "conservation"])


print(fireprot_df.columns)

Index(['DATABASE', 'PROTEIN', 'UniProt_ID', 'PDB_wild', 'MUTATED_CHAIN',
       'POSITION', 'WILD_TYPE_RES', 'MUTATED_RES', '∆∆G_H2O_(kcal/mol)',
       '∆Tm_(C)', 'is_curated', 'conservation', 'METHOD', 'MEASURE', 'pH',
       'Tm_(C)', 'notes', 'PubMed_ID', 'datasets'],
      dtype='object')


## Combining Protherm, ThermoMut, and Fireprot Datasets

In [182]:
# concatonate all 3 dataframes together
all_data_df = pd.concat([protherm_df, thermo_df, fireprot_df], join="outer", ignore_index=True)
all_data_df.reset_index(drop=True, inplace=True)

# raw dataset - no duplicates removed 
all_data_df.to_csv("DBdata/all_data_raw.csv", header=True, index=False, index_label=False)

# removes all exact duplicate rows (dataset has 56,502 values after)
print("Raw: ",len(all_data_df))
all_data_df.drop_duplicates(subset=all_data_df.columns.difference(['DATASET']), keep="first", inplace=True)
print("Len:", len(all_data_df))

# adding WILD_TYPE_RES, MUTATED_RES, and POSITION columns to combined dataset
all_data_df["WILD_TYPE_RES"] = np.nan
all_data_df["MUTATED_RES"] = np.nan
all_data_df["POSITION"] = np.nan

# temporarily replace NaN with "-" to avoid errors
all_data_df["MUTATION"] = all_data_df["MUTATION"].fillna("-")

# replace "," so that all multiple points mutations are split by a single space in MUTATION col
all_data_df["MUTATION"] = all_data_df["MUTATION"].str.replace(",", " ")
all_data_df.reset_index(drop=True, inplace=True)

fixed_space_col = []
for string in all_data_df["MUTATION"]:
    fixed_space_col.append(' '.join(string.split()))
all_data_df["MUTATION"] = fixed_space_col
split_mutation_col = all_data_df["MUTATION"].str.split(" ")

# extricating wild-type (first letter), mutated residue (last letter), and position (inner number) // eg. (L18G)
# from cleaned mutation data

# returns list with wild-type residue (first letter) from mutation
def get_wild_type(split_mutation_column):
    wild_type_list = []
    w_letters = []
    for string in split_mutation_column:
        if "wild-type" in string[0]:
            wild_type = "wild_type"
        elif "-" in string[0] or len(string) == 0:
            wild_type = np.nan
        else:
            for val in string:
                mutation_name = val.strip(" ")
                w_letters.append(mutation_name[0])
                wild_type = ",".join(w_letters)
        wild_type_list.append(wild_type)
        w_letters.clear()
    return wild_type_list

# returns list with mutated residue (last letter) from mutation
def get_mutation_type(split_mutation_column):
    mutation_list = []
    m_letters = []
    for string in split_mutation_column:
        if "wild-type" in string[0]:
            mutation = "wild-type"
        elif "-" in string[0] or len(string) == 0:
            mutation = np.nan
        else:
            for val in string:
                mutation_name = val.strip(" ")
                m_letters.append(mutation_name[-1])
                mutation = ",".join(m_letters)
        mutation_list.append(mutation)
        m_letters.clear()
    return mutation_list

# returns list with position (number) from mutation
def get_position(split_mutation_column):
    position_list = []
    p_letters = []
    for string in split_mutation_column:
        if "wild-type" in string[0]:
            position = "wild-type"
        elif "-" in string[0] or len(string) == 0:
            position = np.nan
        else:
            for val in string:
                mutation_name = val.strip(" ")
                p_letters.append(mutation_name[1:-1])
                position = ",".join(p_letters)
        position_list.append(position)
        p_letters.clear()
    return(position_list)

# adding data to dataframe
all_data_df["WILD_TYPE_RES"] = get_wild_type(split_mutation_col)
all_data_df["MUTATED_RES"] = get_mutation_type(split_mutation_col)
all_data_df["POSITION"] = get_position(split_mutation_col)

all_data_df["WILD_TYPE_RES"] = all_data_df["WILD_TYPE_RES"].str.upper()
all_data_df["MUTATED_RES"] = all_data_df["MUTATED_RES"].str.upper()

# replacing all "-" w/ NaN
all_data_df = all_data_df.replace("-", np.nan)

# cleaning dataframe
all_data_df_clean = all_data_df.copy()

# creates seperate dataframe of all duplicated rows
dup_df = all_data_df_clean[all_data_df_clean.duplicated(subset=["PROTEIN", "MUTATED_RES", "POSITION", "WILD_TYPE_RES", "pH", "T_(C)", "PubMed_ID"])]

# drops duplicated rows from df in order to add back only the rows with the most info
all_data_df_clean = all_data_df_clean[~all_data_df_clean.isin(dup_df)].dropna(how = "all")

# sorting_cols determines duplicated rows
sorting_cols = ["PROTEIN", "MUTATED_RES", "POSITION", "WILD_TYPE_RES", "pH", "T_(C)", "PubMed_ID"]
other_cols = dup_df.columns.difference(sorting_cols)

# cleaned duplicate df with only rows with most info
dup_df = (dup_df.assign(counts=dup_df.count(axis=1))
          .sort_values(["PROTEIN", "MUTATED_RES", "POSITION", "WILD_TYPE_RES", "pH", "T_(C)", "PubMed_ID", 'counts'])
          .drop_duplicates(sorting_cols, keep='last')
          .drop('counts', axis=1))

# add back rows with most info to df
all_data_df_clean = all_data_df_clean.append(dup_df)
print("Clean df length: " + str(len(all_data_df_clean)))

# recasting to ensure data types are correct
all_data_df_clean = cast_to_float(all_data_df_clean, ["pH", "T_(C)", "Tm_(C)", "∆Tm_(C)", "∆H_(kcal/mol)",
                                          "∆G_(kcal/mol)", "∆∆G_(kcal/mol)", "∆∆G_H2O_(kcal/mol)",
                                          "PubMed_ID", "conservation"])

all_data_df_clean = cast_to_string(all_data_df_clean, ["DATABASE", "PROTEIN", "UniProt_ID", "MUTATION", "SOURCE",
                                          "PDB_wild", "PDB_Chain_Mutation", "STATE",
                                          "REVERSIBILITY", "KINGDOM", "PDB_mutant",
                                          "MUTATED_CHAIN", "MEASURE", "METHOD", "POSITION",
                                          "WILD_TYPE_RES", "MUTATED_RES", "is_curated", "datasets"])

# renaming col names for consistency + readibility
names = {"UniProt_ID": "UNIPROT_ID",
         "PDB_wild": "PBD_WILD",
         "PDB_Chain_Mutation": "PBD_CHAIN_MUTATION",
         "PDB_mutant": "PBD_MUTANT",
         "is_curated": "IS_CURATED",
         "datasets": "DATASETS",
         "PubMed_ID": "PUBMED_ID",
         "conservation": "CONSERVATION",
         "∆Tm_(C)": "dTm_(C)",
         "∆H_(kcal/mol)": "dH_(kcal/mol)",
         "∆G_(kcal/mol)": "dG_(kcal/mol)",
         "∆∆G_(kcal/mol)": "ddG_(kcal/mol)",
         "∆∆G_H2O_(kcal/mol)": "ddG_H2O_(kcal/mol)",
         "notes": "NOTES"}

# reordering columns for readibility
column_order = ["DATABASE", "PROTEIN", "UNIPROT_ID", "MUTATION", ""]

all_data_df_clean.rename(columns=names, inplace = True)

all_data_df_clean.to_csv("DBdata/all_data_clean.csv", header=True, index=False, index_label=False)

print(all_data_df_clean.columns)

Raw:  60620
Len: 56541
Clean df length: 37764
Index(['DATABASE', 'PROTEIN', 'UNIPROT_ID', 'MUTATION', 'SOURCE', 'PBD_WILD',
       'PBD_CHAIN_MUTATION', 'pH', 'T_(C)', 'Tm_(C)', 'dTm_(C)',
       'dH_(kcal/mol)', 'dG_(kcal/mol)', 'ddG_(kcal/mol)',
       'ddG_H2O_(kcal/mol)', 'STATE', 'REVERSIBILITY', 'PUBMED_ID',
       'REFERENCE', 'MUTATED_CHAIN', 'KINGDOM', 'PBD_MUTANT', 'MEASURE',
       'METHOD', 'POSITION', 'WILD_TYPE_RES', 'MUTATED_RES', 'IS_CURATED',
       'CONSERVATION', 'NOTES', 'DATASETS'],
      dtype='object')


In [67]:
# fixing individual errors

all_data_df_clean.iloc[1306, 3] = "I3C"
all_data_df_clean.iloc[1306, 26] = "3"
all_data_df_clean.iloc[1306, 27] = "I"
all_data_df_clean.iloc[1306, 28] = "C"



#### Adding boolean "Enzyme" column to indicate if protein is enzyme

In [70]:
# add enzyme column (version using UniProtID's)

# load column from UniProt datbase w/ all enzyme ID's (35 mil entries)
uniprot_35_path = os.path.join("DBdata", "uniprot-v3.tsv")
uniprot_35_ids = pd.read_csv(uniprot_35_path, usecols=["Entry"], sep='\t')

all_data_enzyme_df = all_data_df_clean.copy()

# comparing UniProt ID to find if protein is enzyme
# if enzyme, adds TRUE // if not, adds FALSE // if there is no UniProt ID adds NaN
enzyme = []
i = 0
for entry in all_data_enzyme_df.iloc[:,2]:
    if entry in uniprot_35_ids["Entry"].array:
        enzyme.append(True)
    else:
        if pd.isna(entry):
            enzyme.append(np.nan)
        else:
            enzyme.append(False)
    i = i + 1
    print(i)
    
all_data_enzyme_df["ENZYME"] = enzyme

all_data_enzyme_df.to_csv("DBdata/all_data_clean_enzyme.csv", header=True, index=False, index_label=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

# Reaction Databases

## MuteinDB

Number of Entries: 485

In [383]:
mutein_path = os.path.join("DBdata", "muteindb.csv")
mutein_df = pd.read_csv(mutein_path, sep=",", dtype="unicode")

#print(mutein_df.head())
col_to_drop = ["NO", "Unnamed: 1", "Gen Bank Id", "Pdb Id", "Contact", "pH Opt Value", "Temp opt Value [C]",
                "Signal Seq C", "Signal Seq N", "Stability +4", "Stability -20", "Stability Rt", "Temp Comment",
                "pH Comment", "Unnamed: 42" ]
mutein_df = mutein_df.drop(columns=col_to_drop)

# rename columns
dictionary = {"Uni Prot Id": "UniProt_ID",
              "Temp pH [C]": "Temp_(C)"}

mutein_df.rename(columns=dictionary, inplace=True)

# add database label
mutein_df.insert(0, "Database", 'Mutein')
print(mutein_df.columns)
print(len(mutein_df))

# extricating wild-type (first letter), mutation (last letter), and position (inner #) from "Mutations" data
mutein_df["Wild_type_res"] = np.nan
mutein_df["Mutated_res"] = np.nan
mutein_df["Position"] = np.nan

# temporarily replace NaN with "-" to avoid errors
mutein_df["Mutations"] = mutein_df["Mutations"].fillna("-")

# replace ";" so that all multiple points mutations are split by a single space in MUTATION col
mutein_df["Mutations"] = mutein_df["Mutations"].str.replace(";", " ")

fixed_space_col = []
for string in mutein_df["Mutations"]:
    fixed_space_col.append(' '.join(string.split()))
mutein_df["Mutations"] = fixed_space_col
split_mutation_col = mutein_df["Mutations"].str.split(" ")

mutein_df["Wild_type_res"] = get_wild_type(split_mutation_col)
mutein_df["Mutated_res"] = get_mutation_type(split_mutation_col)
mutein_df["Position"] = get_position(split_mutation_col)

# replacing all "-" w/ NaN
mutein_df = mutein_df.replace("-", np.nan)

# clean (KM) from "K [µM]" list
mutein_df["K [µM]"] = mutein_df["K [µM]"].fillna("-")
split_k_col = mutein_df["K [µM]"].str.split("(")
values = []
for string in split_k_col:
    val = string[0].strip(" ")
    values.append(val)
mutein_df["K [µM]"] = values 

mutein_df = mutein_df.replace("-", np.nan)

# cast type to float
mutein_df = cast_to_float(mutein_df, ["Temp_(C)", "pH Max", "pH Min", "pH Temp [C]", "Temp Max [C]",
                                      "Mol Weight[g/mol]", "Num Mutations", "K [µM]"])

print(mutein_df["K [µM]"])


Index(['Database', 'Wild Type', 'Name', 'Mutations', 'Reaction Type',
       'Substrate', 'Product', 'UniProt_ID', 'pH Max', 'pH Min', 'pH Temp [C]',
       'Temp Max [C]', 'Temp Min [C]', 'Temp_(C)', 'Mol Weight[g/mol]',
       'Num Mutations', 'Organism', 'Tissue', 'pH Buffer', 'Temp Buffer',
       'Kingdom', 'K [µM]', 'Activity Value', 'Activity Unit',
       'Relative activity [%]', 'Activity Type', 'Inhibitor',
       'Expression Host', '# Activities', 'Publication'],
      dtype='object')
485
0        288.000
1         40.800
2        141.400
3         55.000
4         39.500
5            NaN
6         12.800
7            NaN
8       8000.000
9            NaN
10      2200.000
11      1200.000
12      1500.000
13      1900.000
14       800.000
15      3900.000
16      2000.000
17           NaN
18           NaN
19           NaN
20           NaN
21           NaN
22           NaN
23           NaN
24           NaN
25           NaN
26           NaN
27           NaN
28           NaN
29

#### Metadata

In [273]:
print(mutein_df.value_counts("Wild Type"))

Wild Type
CYP102A1                195
CYP3A4                  120
CYP2D6                   78
PAMO                     31
HRP C1                   12
Nitrilase, arylaceto     10
Nitrilase 2               8
MaCel7B                   4
TeCel7A                   4
HiCel7B                   2
P3H type2                 1
dtype: int64
